In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
np.random.seed(123)

In [5]:
train_path = os.path.join('data', 'application_train.csv')
train = pd.read_csv(train_path, index_col='SK_ID_CURR')
train.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.info()

In [ ]:
categ_columns = list(train.select_dtypes(include='object').columns)
categ_columns

In [ ]:
from ipywidgets import interact, fixed
def plot_categ_interact(x, y, data):
    print(x)
    percent_nans = np.round(100*np.sum(data[x].isnull())/data.shape[0], 2)
    if percent_nans >0:
        print('the col {} has {} percent nans in the data'.format(x, percent_nans))
#     return sns.stripplot(x=x, y=y, data=data, jitter=True)
    p = sns.countplot(x, hue=y, data=data);
    p.set_xticklabels(p.get_xticklabels(), rotation=45)
    return p


interact(plot_categ_interact, x=categ_columns, y=fixed('TARGET'), data=fixed(train));

In the column CODE_GENDER the value XNA looks weird. (4 instances, we can drop the or make them nans).

'NAME_EDUCATION_TYPE' and 'WEEKDAY_APPR_PROCESS_START' can be ordinal.

In the column 'NAME_FAMILY_STATUS' we can make more features, like 'Was/Is married'.

'ORGANIZATION_TYPE' needs a further look.

'FONDKAPREMONT_MODE' this is mainly nan, maybe it's not so good. for now, all nan are same as "not specified".

'EMERGENCYSTATE_MODE' maybe all the nan's are 'No'. For now I'll leave it as nan

In [ ]:
# train.ORGANIZATION_TYPE.value_counts()
# list(train.NAME_FAMILY_STATUS.value_counts(dropna=False).index)

In [18]:
def get_ORGANIZATION_TYPE_higher_level(value):
    for t in ['Business Entity', 'Trade', 'Transport', 'Industry']:
        if t in value:
            return t

def col_convert(in_data):
    data = in_data.copy()
    data['Was_Married'] = data.NAME_FAMILY_STATUS.apply(lambda v: v in ['Separated', 'Widow'])
    data['IS_Married'] = data.NAME_FAMILY_STATUS.apply(lambda v: v in ['Married', 'Civil marriage'])
    
    
    data['FONDKAPREMONT_MODE'] = data['FONDKAPREMONT_MODE'].fillna('not specified')
    data['FLAG_OWN_REALTY'] = data['FLAG_OWN_REALTY'].apply(lambda v: v=='Y')
    data['FLAG_OWN_CAR'] = data['FLAG_OWN_CAR'].apply(lambda v: v=='Y')

    data['CODE_GENDER'] = data['CODE_GENDER'].replace({'M':True, 'F':False, 'XNA':None})
    data['NAME_CONTRACT_TYPE'] = data['NAME_CONTRACT_TYPE'].apply(lambda v: v=='Cash loans')
    data['EMERGENCYSTATE_MODE'] = data['EMERGENCYSTATE_MODE'].apply(lambda v: v=='Yes')
    
    data['NAME_EDUCATION_TYPE'] = data['NAME_EDUCATION_TYPE'].replace({'Secondary / secondary special':2,
                                                                       'Higher education':4, 'Incomplete higher':3,
                                                                       'Lower secondary':1, 'Academic degree':5})
    
    data['IS_WEEKEND_APPR_PROCESS_START'] = data['WEEKDAY_APPR_PROCESS_START'].replace({'TUESDAY':0, 'WEDNESDAY':0, 
                                                                                     'MONDAY':0, 'THURSDAY':0,
                                                                                     'FRIDAY':1, 'SATURDAY':1,
                                                                                     'SUNDAY':1}) 
    
    data['WEEKDAY_APPR_PROCESS_START'] = data['WEEKDAY_APPR_PROCESS_START'].replace({'TUESDAY':3, 'WEDNESDAY':4, 
                                                                                     'MONDAY':2, 'THURSDAY':3,
                                                                                     'FRIDAY':6, 'SATURDAY':7,
                                                                                     'SUNDAY':1}) #Im not sure this is right
    
    data['ORGANIZATION_TYPE_upper_level'] = data['ORGANIZATION_TYPE'].apply(get_ORGANIZATION_TYPE_higher_level)
    
    data['EXT_SOURCE_sum'] = data[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
    data['EXT_SOURCE_mean'] = data[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
    data['DOCUMENT_count'] = data[['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
                                   'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
                                   'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16',
                                   'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',]].sum(axis=1)

    return data

In [22]:
neg_pos = np.sum(train.TARGET == 0) / np.sum(train.TARGET == 1)
neg_pos

11.387150050352467

In [ ]:
sns.countplot(train.TARGET); # The data is very unballanced
# np.sum(train.TARGET == 1) / train.TARGET.shape[0]


# "Bureau" Table

In [ ]:
bureau_path = os.path.join('data', 'bureau.csv')
bureau = pd.read_csv(bureau_path, index_col='SK_ID_CURR', dtype={'SK_ID_BUREAU':'category'})
bureau.head()

In [ ]:
# print(bureau['CREDIT_CURRENCY'].value_counts())
pd.get_dummies(bureau['CREDIT_CURRENCY'], prefix='CREDIT_CURRENCY').groupby('SK_ID_CURR').sum().describe()

In [ ]:
bureau.info()

In [ ]:
def count_existance(row, new_table):
    if set(row.index) in list(new_table.index):
        return new_table.loc[row.index, :].shape[0]
    return 0

def add_table_summerized(data, new_table, table_name):
    df = data.copy()
    df['count_{}_instances'.format(table_name)] = df.apply(lambda row: count_existance(row, new_table))
    for num_col in new_table.select_dtypes(exclude=['object', 'category']).columns:
        df = df.join(new_table.groupby(new_table.index.name)[num_col].sum().rename('{}_sum'.format(num_col)),rsuffix=table_name)
        df = df.join(new_table.groupby(new_table.index.name)[num_col].mean().rename('{}_mean'.format(num_col)),rsuffix=table_name)
        df = df.join(new_table.groupby(new_table.index.name)[num_col].median().rename('{}_median'.format(num_col)),rsuffix=table_name)
        df = df.join(new_table.groupby(new_table.index.name)[num_col].std().rename('{}_std'.format(num_col)),rsuffix=table_name)
    
    for categ_col in new_table.select_dtypes(include='object').columns:
        dummie_cols = pd.get_dummies(new_table[categ_col], prefix=categ_col).groupby(new_table.index.name).sum()
        df = df.join(dummie_cols, rsuffix=table_name)
        
#         dummies_percent = dummie_cols/df['count_{}_instances'.format(table_name)]
#         for col in dummies_percent.columns:
#             dummies_precent[col].rename('{}_percent'.format(col), inplace=True)
#         df = df.join(dummies_percent, rsuffix=table_name)
    
    
    return df

# "Bureau balance" Table

In [ ]:
bureau_balance_path = os.path.join('data', 'bureau_balance.csv')
bureau_balance = pd.read_csv(bureau_balance_path, index_col='SK_ID_BUREAU')
bureau_balance.head()

In [ ]:
bureau = add_table_summerized(bureau, bureau_balance, 'Bureau balance')

# "Bureau balance" Table

In [2]:
previous_application_path = os.path.join('data', 'previous_application.csv')
previous_application = pd.read_csv(previous_application_path, index_col='SK_ID_CURR')
previous_application.head()

/home/ariel/venv/lib/python3.5/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,SK_ID_PREV,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
271877,2030495,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
108129,2802425,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
122040,2523466,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
176158,2819243,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
202054,1784265,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
previous_application.NAME_YIELD_GROUP.value_counts()
previous_application['NAME_YIELD_GROUP'] = previous_application['NAME_YIELD_GROUP'].replace({'XNA':0, })

XNA           517215
middle        385532
high          353331
low_normal    322095
low_action     92041
Name: NAME_YIELD_GROUP, dtype: int64

# Predict

In [19]:
from sklearn import metrics
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import Imputer, StandardScaler, RobustScaler
import xgboost as xgb
# from sklearn.svm import SVR, LinearSVR
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
# from scipy.special import boxcox1p, inv_boxcox1p
import datetime
# from imblearn import combine, over_sampling
# from imblearn import pipeline as imblearn_pipeline
from imblearn.ensemble import BalancedBaggingClassifier

import category_encoders
np.random.seed(123)


In [ ]:
import multiprocessing
def model_run(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train) 
    test_pred = model.predict(X_test)

    train_auc = metrics.roc_auc_score(y_train, train_pred)
    test_auc = metrics.roc_auc_score(y_test, test_pred)

    return train_auc, test_auc

def test_on_train(model, X, y):
    input_to_multi = []
    start = datetime.datetime.now()
    for train_index, test_index in RepeatedStratifiedKFold(n_splits=5, n_repeats=4, random_state=222).split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        input_to_multi.append([model, X_train, X_test, y_train, y_test])

    with multiprocessing.Pool() as p:
        KFold_results = p.starmap(model_run, input_to_multi)


    print(datetime.datetime.now() - start)
    return pd.DataFrame(KFold_results,
                        columns=['train_auc', 'test_auc']).mean(axis=0)


In [ ]:
def test_model(X, y, model):
    start = datetime.datetime.now()
    train_auc = []
    test_auc = []
    for train_index, test_index in RepeatedStratifiedKFold(n_splits=5, n_repeats=4, random_state=123).split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        
        model.fit(X_train, y_train)
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
        
        train = metrics.roc_auc_score(y_train, train_pred)
        test = metrics.roc_auc_score(y_test, test_pred)
        
        train_auc.append(train)
        test_auc.append(test)
        

    print(datetime.datetime.now() - start) 
    return train_auc, test_auc

In [20]:
np.random.seed(123)
start = datetime.datetime.now()
print(start)
X = col_convert(train).drop('TARGET', axis=1)
y = train['TARGET']

# X = add_bureau_summerized(X, bureau, 'bureau')
print(datetime.datetime.now() - start)
 


categorical_cols = list(X.select_dtypes(include='object').columns)
print(categorical_cols)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

2018-07-14 16:04:41.344074
0:00:02.204058
['CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'ORGANIZATION_TYPE_upper_level']


In [24]:
len(list(X.columns))

127

In [ ]:
np.random.seed(123)
start = datetime.datetime.now()
model = Pipeline([
# model = imblearn_pipeline.Pipeline([
    ('categ', category_encoders.binary.BinaryEncoder(cols=categorical_cols,
                                                      drop_invariant=True, impute_missing=True)),
    ('impute', Imputer()),
    ('scale', StandardScaler()),
# #     ('resample', combine.SMOTEENN(random_state=123)),
# #     ('resample', combine.SMOTETomek()),
    ('xgb', xgb.XGBClassifier(eval_metric='auc', learning_rate=0.1, max_depth=6, booster='dart',reg_lambda=0.1,
#                               sample_type='weighted',
                              scale_pos_weight=neg_pos))
#     ('resample', over_sampling.SMOTE()),
#     ('logreg', LogisticRegression())
#     ('resample_Ensemble', BalancedBaggingClassifier(base_estimator=LogisticRegression(), n_estimators=10,
#                                                     max_samples=1.0, max_features=1.0, bootstrap=True,
#                                                     bootstrap_features=False, oob_score=False, warm_start=False,
#                                                     ratio='auto', replacement=False, n_jobs=3, random_state=123))
])

# print(test_on_train(model, X, y))
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

print(metrics.roc_auc_score(y_train, train_pred))
print(metrics.roc_auc_score(y_test, test_pred))
print(metrics.classification_report(y_test, test_pred))
print(datetime.datetime.now() - start)

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
#     'xgb__max_depth':np.arange(4, 9),
#     'xgb__normalize_type':['tree', 'forest'],
    'xgb__sample_type':['uniform', 'weighted'],
    'xgb__reg_lambda':[0.4, 0.8, 1, 1.5, 2],
#          'xgb__colsample_bytree':np.arange(0.7, 1.0, 0.1),
#          'xgb__colsample_bylevel':np.arange(0.7, 1.0, 0.1),
    'xgb__objective':['reg:logistic', 'binary:logistic', 'binary:logitraw'],
#          'xgb__booster':['dart', 'gbtree', 'gblinear']
}
GCV = GridSearchCV(model, params, cv=4, verbose=2, n_jobs=3)
GCV.fit(X, y)

# predict on real Test

In [ ]:
test = pd.read_csv(os.path.join('data', 'application_test.csv'), index_col='SK_ID_CURR')
test.head()

In [ ]:
model.fit(X, y)
all_train_pred = model.predict(X) 
print(metrics.roc_auc_score(y, all_train_pred))

In [ ]:
test_pred = model.predict_proba(col_convert(test))

In [ ]:
submission = pd.DataFrame(list(zip(list(test.index) ,test_pred[:, 1])),
                          columns = ['SK_ID_CURR', 'TARGET']).set_index('SK_ID_CURR')
submission.head()

In [ ]:
submission.to_csv(os.path.join('Submission_files', 'submission_{}.csv'.format(str(datetime.datetime.now()).replace(' ', '-'))))